In [1]:
import os
 
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        continue
        #print(root) #当前目录路径
        #print(dirs) #当前路径下所有子目录
       # print(files) #当前路径下所有非目录子文件
    return files


## 控制c 只有需要检测的图片

# 加载模型

In [2]:
import argparse
import os
import torch
import torch.backends.cudnn as cudnn
import numpy as np
import PIL.Image as pil_image

from myutils import rgb2ycbcr, ycbcr2rgb, calc_psnr

In [3]:
cudnn.benchmark = True
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
from rcan import make_model

In [5]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=[2]
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1

In [6]:
model = make_model(modelargs()).to(device)

# 定义psnr函数

In [7]:
def get_psnr(testdata,myscale,picidx,j):
   # print(filename)
    scale=myscale
   # text= os.path.split(filename)[-1]
  #  textname = os.path.splitext(text)[0]
    
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    
    clrpath=file_name(lrpath)
    chrpath=file_name(hrpath)
  #  print(clrpath[picidx])
    textname=str(clrpath[picidx]).split("_LRBI")[0]
#     textname=textnames[0]
#     for i in range(len(textnames)-1):
#         if(textnames[i+1]=='LRBI'):
#             break
#         textname=textname+textnames[i+1]
        
        
    
    
    
    lr = pil_image.open(lrpath+clrpath[picidx]).convert('RGB')
    lr = np.array(lr).astype(np.float32).transpose([2,0,1])
 #   lrycbcr = rgb2ycbcr(lr)
    
   # y = lrycbcr[..., 0]
   # print(lr.shape)
    y =lr /255.
    y = torch.from_numpy(y).to(device)
    y = y.unsqueeze(0)
    
    
    #print("y:{}".format(y.size()))########################################################################

    hr = pil_image.open(hrpath+chrpath[picidx]).convert('RGB')
    hr=np.array(hr).astype(np.float32).transpose([2,0,1])
  #  hrycbcrb = rgb2ycbcr(hr)
  #  ya = hrycbcrb[..., 0]
  
    ya =hr/ 255.
    ya = torch.from_numpy(ya).to(device)
    ya = ya.unsqueeze(0)
  #  print("ya:{}".format(ya.size()))########################################################################

    #print(y.size())

    with torch.no_grad():
        preds = model(y).clamp(0.0, 1.0)

    psnr = calc_psnr(ya, preds)

    print('PSNR: {:.2f}'.format(psnr))

    preds = preds.mul(255.0).cpu().numpy().squeeze(0)
    
    #print(preds.shape)
    output = np.array(preds).transpose([1, 2, 0])
    output = np.clip((output), 0.0, 255.0).astype(np.uint8)
    output = pil_image.fromarray(output)
    savepath='./save/{}/{}/x{}/{}_idn_x{}.png'.format(j,testdata,myscale,textname,scale)
    
    if not os.path.exists(os.path.split(savepath)[0]):
        os.makedirs(os.path.split(savepath)[0])
     
    
    output.save(savepath)
    
    preds = None
    output = None
    ya = None
    y = None
    lr = None
    hr = None
    
    
    
    return psnr


# 单独计算psnr

In [8]:
modelname='./checkpoint/model_epoch_900.pth'
print(modelname)

./checkpoint/model_epoch_900.pth


In [9]:
from model import IDN

In [10]:
class modelargs():
    def __init__(self):
        self.n_resgroups=10
        self.n_resblocks=20
        self.n_feats=64
        self.reduction=16
        self.scale=2
        self.rgb_range=255
        self.n_colors=3
        self.res_scale=1
        self.d=16
        self.s=4

In [11]:
model = IDN(modelargs())


In [12]:
allpsnr=[]

In [13]:
for j in range(1000):
    modelname='./checkpoint/model_epoch_{}.pth'.format(j+1)
    print(modelname)
    state_dict = model.state_dict()
    for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

    testdata="Set5"
    myscale = 2
    hrpath="../HR/{}/x{}/".format(testdata,myscale)
    lrpath="../LR/{}/x{}/".format(testdata,myscale)
    clrpath=file_name(lrpath)
    count=len(clrpath)

    hrpath="../HR/{}/x{}/".format(testdata,myscale)

    chrpath=file_name(hrpath)

    count=len(chrpath)
    print(count)
    model=model.to(device)
    model.eval()
    sum_psnr=0
    for i in range(count):
     #   filename = os.path.join("../"+testdata+"/"+c[i])

        sum_psnr += get_psnr(testdata,myscale,i,j)

    print("avg_psnr is :{:.2f}".format(sum_psnr/count))
    allpsnr.append(sum_psnr/count)

./checkpoint/model_epoch_1.pth
5
PSNR: 5.22
PSNR: 4.16
PSNR: 4.69
PSNR: 5.37
PSNR: 5.32
avg_psnr is :4.95
./checkpoint/model_epoch_2.pth
5
PSNR: 5.95
PSNR: 5.13
PSNR: 5.32
PSNR: 6.57
PSNR: 5.76
avg_psnr is :5.75
./checkpoint/model_epoch_3.pth
5
PSNR: 6.34
PSNR: 5.97
PSNR: 5.62
PSNR: 7.25
PSNR: 5.94
avg_psnr is :6.22
./checkpoint/model_epoch_4.pth
5
PSNR: 6.97
PSNR: 6.13
PSNR: 5.63
PSNR: 7.65
PSNR: 6.25
avg_psnr is :6.53
./checkpoint/model_epoch_5.pth
5
PSNR: 7.37
PSNR: 6.33
PSNR: 5.71
PSNR: 8.00
PSNR: 6.55
avg_psnr is :6.79
./checkpoint/model_epoch_6.pth
5
PSNR: 7.56
PSNR: 6.37
PSNR: 5.73
PSNR: 8.26
PSNR: 6.76
avg_psnr is :6.94
./checkpoint/model_epoch_7.pth
5
PSNR: 7.74
PSNR: 6.49
PSNR: 5.88
PSNR: 8.57
PSNR: 7.01
avg_psnr is :7.14
./checkpoint/model_epoch_8.pth
5
PSNR: 7.92
PSNR: 6.65
PSNR: 5.99
PSNR: 8.90
PSNR: 7.13
avg_psnr is :7.32
./checkpoint/model_epoch_9.pth
5
PSNR: 8.23
PSNR: 6.73
PSNR: 6.04
PSNR: 9.15
PSNR: 7.26
avg_psnr is :7.48
./checkpoint/model_epoch_10.pth
5
PSNR: 8.22
P

PSNR: 16.54
PSNR: 12.95
PSNR: 9.80
PSNR: 16.92
PSNR: 13.03
avg_psnr is :13.85
./checkpoint/model_epoch_76.pth
5
PSNR: 16.79
PSNR: 13.04
PSNR: 9.84
PSNR: 17.08
PSNR: 13.22
avg_psnr is :14.00
./checkpoint/model_epoch_77.pth
5
PSNR: 16.80
PSNR: 13.10
PSNR: 9.90
PSNR: 17.13
PSNR: 13.29
avg_psnr is :14.04
./checkpoint/model_epoch_78.pth
5
PSNR: 16.76
PSNR: 13.22
PSNR: 9.95
PSNR: 17.15
PSNR: 13.33
avg_psnr is :14.08
./checkpoint/model_epoch_79.pth
5
PSNR: 17.00
PSNR: 13.25
PSNR: 10.01
PSNR: 17.29
PSNR: 13.42
avg_psnr is :14.19
./checkpoint/model_epoch_80.pth
5
PSNR: 16.93
PSNR: 13.28
PSNR: 10.06
PSNR: 17.40
PSNR: 13.49
avg_psnr is :14.23
./checkpoint/model_epoch_81.pth
5
PSNR: 17.14
PSNR: 13.38
PSNR: 10.10
PSNR: 17.49
PSNR: 13.58
avg_psnr is :14.34
./checkpoint/model_epoch_82.pth
5
PSNR: 17.35
PSNR: 13.48
PSNR: 10.20
PSNR: 17.58
PSNR: 13.64
avg_psnr is :14.45
./checkpoint/model_epoch_83.pth
5
PSNR: 17.19
PSNR: 13.47
PSNR: 10.20
PSNR: 17.52
PSNR: 13.60
avg_psnr is :14.40
./checkpoint/model_ep

PSNR: 17.00
PSNR: 13.59
PSNR: 21.32
PSNR: 17.42
avg_psnr is :18.25
./checkpoint/model_epoch_148.pth
5
PSNR: 21.97
PSNR: 17.15
PSNR: 13.69
PSNR: 21.38
PSNR: 17.47
avg_psnr is :18.33
./checkpoint/model_epoch_149.pth
5
PSNR: 22.02
PSNR: 17.13
PSNR: 13.70
PSNR: 21.39
PSNR: 17.50
avg_psnr is :18.35
./checkpoint/model_epoch_150.pth
5
PSNR: 22.02
PSNR: 17.15
PSNR: 13.74
PSNR: 21.33
PSNR: 17.50
avg_psnr is :18.35
./checkpoint/model_epoch_151.pth
5
PSNR: 21.99
PSNR: 17.24
PSNR: 13.82
PSNR: 21.51
PSNR: 17.60
avg_psnr is :18.43
./checkpoint/model_epoch_152.pth
5
PSNR: 22.02
PSNR: 17.35
PSNR: 13.89
PSNR: 21.61
PSNR: 17.68
avg_psnr is :18.51
./checkpoint/model_epoch_153.pth
5
PSNR: 22.31
PSNR: 17.26
PSNR: 13.88
PSNR: 21.62
PSNR: 17.74
avg_psnr is :18.56
./checkpoint/model_epoch_154.pth
5
PSNR: 22.32
PSNR: 17.35
PSNR: 13.93
PSNR: 21.71
PSNR: 17.80
avg_psnr is :18.62
./checkpoint/model_epoch_155.pth
5
PSNR: 22.36
PSNR: 17.42
PSNR: 13.98
PSNR: 21.74
PSNR: 17.83
avg_psnr is :18.67
./checkpoint/model_ep

./checkpoint/model_epoch_219.pth
5
PSNR: 25.37
PSNR: 20.37
PSNR: 16.83
PSNR: 24.40
PSNR: 20.79
avg_psnr is :21.55
./checkpoint/model_epoch_220.pth
5
PSNR: 25.56
PSNR: 20.41
PSNR: 16.86
PSNR: 24.44
PSNR: 20.84
avg_psnr is :21.62
./checkpoint/model_epoch_221.pth
5
PSNR: 25.42
PSNR: 20.49
PSNR: 16.89
PSNR: 24.44
PSNR: 20.87
avg_psnr is :21.62
./checkpoint/model_epoch_222.pth
5
PSNR: 25.62
PSNR: 20.49
PSNR: 16.91
PSNR: 24.36
PSNR: 20.86
avg_psnr is :21.65
./checkpoint/model_epoch_223.pth
5
PSNR: 24.72
PSNR: 20.42
PSNR: 16.86
PSNR: 24.29
PSNR: 20.75
avg_psnr is :21.41
./checkpoint/model_epoch_224.pth
5
PSNR: 25.61
PSNR: 20.60
PSNR: 17.04
PSNR: 24.45
PSNR: 20.95
avg_psnr is :21.73
./checkpoint/model_epoch_225.pth
5
PSNR: 25.70
PSNR: 20.66
PSNR: 17.06
PSNR: 24.60
PSNR: 21.05
avg_psnr is :21.81
./checkpoint/model_epoch_226.pth
5
PSNR: 25.30
PSNR: 20.50
PSNR: 16.96
PSNR: 24.24
PSNR: 20.79
avg_psnr is :21.56
./checkpoint/model_epoch_227.pth
5
PSNR: 25.52
PSNR: 20.69
PSNR: 17.12
PSNR: 24.41
PSNR:

avg_psnr is :24.01
./checkpoint/model_epoch_291.pth
5
PSNR: 27.91
PSNR: 23.09
PSNR: 19.32
PSNR: 26.34
PSNR: 23.28
avg_psnr is :23.99
./checkpoint/model_epoch_292.pth
5
PSNR: 27.95
PSNR: 23.11
PSNR: 19.30
PSNR: 26.50
PSNR: 23.43
avg_psnr is :24.06
./checkpoint/model_epoch_293.pth
5
PSNR: 28.34
PSNR: 23.27
PSNR: 19.41
PSNR: 26.57
PSNR: 23.55
avg_psnr is :24.23
./checkpoint/model_epoch_294.pth
5
PSNR: 28.50
PSNR: 23.31
PSNR: 19.45
PSNR: 26.64
PSNR: 23.62
avg_psnr is :24.30
./checkpoint/model_epoch_295.pth
5
PSNR: 28.54
PSNR: 23.35
PSNR: 19.50
PSNR: 26.70
PSNR: 23.66
avg_psnr is :24.35
./checkpoint/model_epoch_296.pth
5
PSNR: 28.60
PSNR: 23.38
PSNR: 19.53
PSNR: 26.71
PSNR: 23.72
avg_psnr is :24.39
./checkpoint/model_epoch_297.pth
5
PSNR: 28.56
PSNR: 23.43
PSNR: 19.56
PSNR: 26.72
PSNR: 23.72
avg_psnr is :24.40
./checkpoint/model_epoch_298.pth
5
PSNR: 28.54
PSNR: 23.42
PSNR: 19.57
PSNR: 26.73
PSNR: 23.75
avg_psnr is :24.40
./checkpoint/model_epoch_299.pth
5
PSNR: 28.67
PSNR: 23.47
PSNR: 19.6

...

PSNR: 31.18
PSNR: 30.71
avg_psnr is :31.17
./checkpoint/model_epoch_650.pth
5
PSNR: 35.48
PSNR: 32.54
PSNR: 26.01
PSNR: 31.19
PSNR: 30.73
avg_psnr is :31.19
./checkpoint/model_epoch_651.pth
5
PSNR: 35.47
PSNR: 32.56
PSNR: 26.01
PSNR: 31.20
PSNR: 30.73
avg_psnr is :31.20
./checkpoint/model_epoch_652.pth
5
PSNR: 35.49
PSNR: 32.57
PSNR: 26.02
PSNR: 31.19
PSNR: 30.74
avg_psnr is :31.20
./checkpoint/model_epoch_653.pth
5
PSNR: 35.50
PSNR: 32.60
PSNR: 26.04
PSNR: 31.20
PSNR: 30.75
avg_psnr is :31.22
./checkpoint/model_epoch_654.pth
5
PSNR: 35.50
PSNR: 32.62
PSNR: 26.04
PSNR: 31.20
PSNR: 30.75
avg_psnr is :31.22
./checkpoint/model_epoch_655.pth
5
PSNR: 35.49
PSNR: 32.63
PSNR: 26.05
PSNR: 31.20
PSNR: 30.76
avg_psnr is :31.23
./checkpoint/model_epoch_656.pth
5
PSNR: 35.47
PSNR: 32.65
PSNR: 26.06
PSNR: 31.19
PSNR: 30.76
avg_psnr is :31.22
./checkpoint/model_epoch_657.pth
5
PSNR: 35.49
PSNR: 32.66
PSNR: 26.06
PSNR: 31.21
PSNR: 30.77
avg_psnr is :31.24
./checkpoint/model_epoch_658.pth
5
PSNR: 35.5

PSNR: 33.60
PSNR: 26.48
PSNR: 31.44
PSNR: 31.22
avg_psnr is :31.73
./checkpoint/model_epoch_722.pth
5
PSNR: 35.87
PSNR: 33.61
PSNR: 26.49
PSNR: 31.43
PSNR: 31.22
avg_psnr is :31.72
./checkpoint/model_epoch_723.pth
5
PSNR: 35.89
PSNR: 33.63
PSNR: 26.50
PSNR: 31.44
PSNR: 31.23
avg_psnr is :31.74
./checkpoint/model_epoch_724.pth
5
PSNR: 35.93
PSNR: 33.64
PSNR: 26.51
PSNR: 31.45
PSNR: 31.25
avg_psnr is :31.76
./checkpoint/model_epoch_725.pth
5
PSNR: 35.93
PSNR: 33.65
PSNR: 26.51
PSNR: 31.45
PSNR: 31.25
avg_psnr is :31.76
./checkpoint/model_epoch_726.pth
5
PSNR: 35.95
PSNR: 33.68
PSNR: 26.53
PSNR: 31.46
PSNR: 31.26
avg_psnr is :31.78
./checkpoint/model_epoch_727.pth
5
PSNR: 35.96
PSNR: 33.69
PSNR: 26.53
PSNR: 31.46
PSNR: 31.27
avg_psnr is :31.78
./checkpoint/model_epoch_728.pth
5
PSNR: 35.89
PSNR: 33.69
PSNR: 26.52
PSNR: 31.45
PSNR: 31.25
avg_psnr is :31.76
./checkpoint/model_epoch_729.pth
5
PSNR: 35.95
PSNR: 33.72
PSNR: 26.54
PSNR: 31.46
PSNR: 31.28
avg_psnr is :31.79
./checkpoint/model_ep

./checkpoint/model_epoch_793.pth
5
PSNR: 35.38
PSNR: 34.22
PSNR: 26.77
PSNR: 31.49
PSNR: 31.34
avg_psnr is :31.84
./checkpoint/model_epoch_794.pth
5
PSNR: 35.79
PSNR: 34.30
PSNR: 26.80
PSNR: 31.50
PSNR: 31.47
avg_psnr is :31.97
./checkpoint/model_epoch_795.pth
5
PSNR: 36.04
PSNR: 34.44
PSNR: 26.84
PSNR: 31.59
PSNR: 31.55
avg_psnr is :32.09
./checkpoint/model_epoch_796.pth
5
PSNR: 36.22
PSNR: 34.48
PSNR: 26.85
PSNR: 31.62
PSNR: 31.60
avg_psnr is :32.15
./checkpoint/model_epoch_797.pth
5
PSNR: 36.23
PSNR: 34.50
PSNR: 26.87
PSNR: 31.63
PSNR: 31.61
avg_psnr is :32.17
./checkpoint/model_epoch_798.pth
5
PSNR: 36.25
PSNR: 34.51
PSNR: 26.87
PSNR: 31.63
PSNR: 31.62
avg_psnr is :32.18
./checkpoint/model_epoch_799.pth
5
PSNR: 36.21
PSNR: 34.51
PSNR: 26.88
PSNR: 31.63
PSNR: 31.62
avg_psnr is :32.17
./checkpoint/model_epoch_800.pth
5
PSNR: 36.29
PSNR: 34.55
PSNR: 26.89
PSNR: 31.63
PSNR: 31.64
avg_psnr is :32.20
./checkpoint/model_epoch_801.pth
5
PSNR: 36.29
PSNR: 34.56
PSNR: 26.89
PSNR: 31.64
PSNR:

avg_psnr is :32.47
./checkpoint/model_epoch_865.pth
5
PSNR: 36.46
PSNR: 35.16
PSNR: 27.14
PSNR: 31.73
PSNR: 31.88
avg_psnr is :32.48
./checkpoint/model_epoch_866.pth
5
PSNR: 36.44
PSNR: 35.17
PSNR: 27.14
PSNR: 31.73
PSNR: 31.89
avg_psnr is :32.47
./checkpoint/model_epoch_867.pth
5
PSNR: 36.45
PSNR: 35.17
PSNR: 27.14
PSNR: 31.74
PSNR: 31.89
avg_psnr is :32.48
./checkpoint/model_epoch_868.pth
5
PSNR: 36.51
PSNR: 35.20
PSNR: 27.15
PSNR: 31.75
PSNR: 31.91
avg_psnr is :32.50
./checkpoint/model_epoch_869.pth
5
PSNR: 36.47
PSNR: 35.19
PSNR: 27.16
PSNR: 31.74
PSNR: 31.90
avg_psnr is :32.49
./checkpoint/model_epoch_870.pth
5
PSNR: 36.48
PSNR: 35.19
PSNR: 27.16
PSNR: 31.75
PSNR: 31.90
avg_psnr is :32.50
./checkpoint/model_epoch_871.pth
5
PSNR: 36.50
PSNR: 35.21
PSNR: 27.16
PSNR: 31.75
PSNR: 31.91
avg_psnr is :32.51
./checkpoint/model_epoch_872.pth
5
PSNR: 36.42
PSNR: 35.20
PSNR: 27.16
PSNR: 31.74
PSNR: 31.89
avg_psnr is :32.48
./checkpoint/model_epoch_873.pth
5
PSNR: 36.48
PSNR: 35.22
PSNR: 27.1

PSNR: 31.82
PSNR: 32.10
avg_psnr is :32.72
./checkpoint/model_epoch_937.pth
5
PSNR: 36.63
PSNR: 35.67
PSNR: 27.35
PSNR: 31.83
PSNR: 32.10
avg_psnr is :32.72
./checkpoint/model_epoch_938.pth
5
PSNR: 36.62
PSNR: 35.69
PSNR: 27.35
PSNR: 31.83
PSNR: 32.10
avg_psnr is :32.72
./checkpoint/model_epoch_939.pth
5
PSNR: 36.22
PSNR: 35.57
PSNR: 27.32
PSNR: 31.77
PSNR: 31.97
avg_psnr is :32.57
./checkpoint/model_epoch_940.pth
5
PSNR: 36.60
PSNR: 35.66
PSNR: 27.35
PSNR: 31.83
PSNR: 32.10
avg_psnr is :32.71
./checkpoint/model_epoch_941.pth
5
PSNR: 36.60
PSNR: 35.69
PSNR: 27.35
PSNR: 31.83
PSNR: 32.09
avg_psnr is :32.71
./checkpoint/model_epoch_942.pth
5
PSNR: 36.58
PSNR: 35.67
PSNR: 27.35
PSNR: 31.82
PSNR: 32.09
avg_psnr is :32.70
./checkpoint/model_epoch_943.pth
5
PSNR: 36.56
PSNR: 35.64
PSNR: 27.35
PSNR: 31.83
PSNR: 32.09
avg_psnr is :32.69
./checkpoint/model_epoch_944.pth
5
PSNR: 36.52
PSNR: 35.66
PSNR: 27.35
PSNR: 31.81
PSNR: 32.07
avg_psnr is :32.68
./checkpoint/model_epoch_945.pth
5
PSNR: 36.6

In [15]:
len(allpsnr)

1000

In [16]:
mymax=0
mymaxi=-1
for i in range(len(allpsnr)):
    if(allpsnr[i]>mymax):
        mymax=allpsnr[i]
        mymaxi=i

In [17]:
mymax

tensor(32.8554, device='cuda:0')

In [18]:
mymaxi

996

In [19]:
988

988

In [36]:
max1=mymax

In [37]:
max1

tensor(35.2092, device='cuda:0')

In [20]:
modelname='./checkpoint/model_epoch_900.pth'
print(modelname)
state_dict = model.state_dict()
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
    if n in state_dict.keys():
        state_dict[n].copy_(p)
    else:
        raise KeyError(n)

testdata="Set5"
myscale = 2
hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i,j)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))
allpsnr.append(sum_psnr/count)

./checkpoint/model_epoch_900.pth
5
PSNR: 37.29
PSNR: 39.57
PSNR: 32.49
PSNR: 32.26
PSNR: 34.65
avg_psnr is :35.25


[tensor(35.2499, device='cuda:0'), tensor(35.2499, device='cuda:0')]

In [41]:
testdata="Manga109"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

109


KeyboardInterrupt: 

In [16]:
testdata="Set14"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

14
PSNR: 23.12
PSNR: 27.00
PSNR: 27.34
PSNR: 28.94
PSNR: 25.91
PSNR: 32.00
PSNR: 29.88
PSNR: 32.38
PSNR: 33.32
PSNR: 28.82
PSNR: 33.12
PSNR: 30.91
PSNR: 26.87
PSNR: 31.23
avg_psnr is :29.35


In [17]:
testdata="B100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)
model.load_state_dict(torch.load(modelname))
model=model.to(device)
model.eval()

sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 25.14
PSNR: 29.78
PSNR: 27.52
PSNR: 32.94
PSNR: 28.34
PSNR: 34.79
PSNR: 29.47
PSNR: 25.07
PSNR: 29.97
PSNR: 31.78
PSNR: 25.92
PSNR: 31.14
PSNR: 33.48
PSNR: 30.69
PSNR: 27.18
PSNR: 28.57
PSNR: 35.61
PSNR: 34.27
PSNR: 26.61
PSNR: 27.38
PSNR: 24.26
PSNR: 25.91
PSNR: 25.99
PSNR: 28.27
PSNR: 30.22
PSNR: 29.83
PSNR: 29.91
PSNR: 32.44
PSNR: 32.06
PSNR: 21.30
PSNR: 32.24
PSNR: 22.62
PSNR: 28.23
PSNR: 25.71
PSNR: 35.50
PSNR: 29.00
PSNR: 28.61
PSNR: 29.11
PSNR: 31.04
PSNR: 35.51
PSNR: 28.17
PSNR: 26.86
PSNR: 28.02
PSNR: 29.58
PSNR: 25.95
PSNR: 35.60
PSNR: 24.98
PSNR: 27.21
PSNR: 29.04
PSNR: 33.51
PSNR: 26.42
PSNR: 25.19
PSNR: 33.45
PSNR: 34.04
PSNR: 29.94
PSNR: 27.31
PSNR: 22.48
PSNR: 28.88
PSNR: 33.36
PSNR: 31.61
PSNR: 32.99
PSNR: 30.41
PSNR: 35.47
PSNR: 26.59
PSNR: 27.64
PSNR: 30.68
PSNR: 39.54
PSNR: 23.32
PSNR: 24.08
PSNR: 30.93
PSNR: 30.95
PSNR: 27.76
PSNR: 29.24
PSNR: 27.70
PSNR: 25.54
PSNR: 32.07
PSNR: 25.47
PSNR: 31.87
PSNR: 31.52
PSNR: 31.74
PSNR: 37.45
PSNR: 29.39
PSNR: 23.88


In [18]:
testdata="Urban100"


hrpath="../HR/{}/x{}/".format(testdata,myscale)
lrpath="../LR/{}/x{}/".format(testdata,myscale)
clrpath=file_name(lrpath)
count=len(clrpath)

hrpath="../HR/{}/x{}/".format(testdata,myscale)

chrpath=file_name(hrpath)

count=len(chrpath)
print(count)


model.eval()
sum_psnr=0
for i in range(count):
 #   filename = os.path.join("../"+testdata+"/"+c[i])
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("avg_psnr is :{:.2f}".format(sum_psnr/count))

100
PSNR: 28.69
PSNR: 27.44
PSNR: 25.46
PSNR: 23.44
PSNR: 25.97
PSNR: 24.22
PSNR: 28.35
PSNR: 23.33
PSNR: 33.23
PSNR: 27.56
PSNR: 18.08
PSNR: 25.85
PSNR: 28.91
PSNR: 26.59
PSNR: 27.13
PSNR: 28.48
PSNR: 26.20
PSNR: 27.59
PSNR: 22.67
PSNR: 23.22
PSNR: 30.14
PSNR: 29.28
PSNR: 29.22
PSNR: 20.77
PSNR: 29.55
PSNR: 29.32
PSNR: 30.50
PSNR: 31.43
PSNR: 26.27
PSNR: 24.17
PSNR: 26.31
PSNR: 29.04
PSNR: 27.69
PSNR: 24.60
PSNR: 26.88
PSNR: 28.23
PSNR: 25.81
PSNR: 28.11
PSNR: 25.15
PSNR: 22.29
PSNR: 23.66
PSNR: 27.50
PSNR: 25.59
PSNR: 27.99
PSNR: 23.63
PSNR: 26.31
PSNR: 23.66
PSNR: 21.56
PSNR: 23.50
PSNR: 26.75
PSNR: 27.36
PSNR: 26.70
PSNR: 23.48
PSNR: 23.60
PSNR: 26.58
PSNR: 25.09
PSNR: 28.98
PSNR: 25.06
PSNR: 24.59
PSNR: 24.78
PSNR: 25.91
PSNR: 22.91
PSNR: 20.60
PSNR: 27.77
PSNR: 26.26
PSNR: 25.34
PSNR: 20.06
PSNR: 29.79
PSNR: 26.13
PSNR: 26.44
PSNR: 29.06
PSNR: 20.02
PSNR: 22.05
PSNR: 25.38
PSNR: 29.76
PSNR: 23.66
PSNR: 26.30
PSNR: 28.70
PSNR: 26.94
PSNR: 34.99
PSNR: 31.08
PSNR: 30.27
PSNR: 24.17


In [13]:
myscale = 3
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
   
       
    sum_psnr += get_psnr(testdata,myscale,i)

print("psnr is :{:.2f}".format(sum_psnr/count))

PSNR: 30.63
PSNR: 25.66
PSNR: 34.00
PSNR: 33.36
PSNR: 27.44
PSNR: 28.82
PSNR: 27.34
PSNR: 27.23
PSNR: 26.26


KeyboardInterrupt: 

In [ ]:
myscale = 4
for n, p in torch.load(modelname, map_location=lambda storage, loc: storage).items():
        if n in state_dict.keys():
            state_dict[n].copy_(p)
        else:
            raise KeyError(n)

model.eval()
sum_psnr=0
for i in range(count):
      
    sum_psnr += get_psnr(testdata,myscale,i)
print("psnr is :{:.2f}".format(sum_psnr/count))